# __Code__

In [ ]:
import os
import requests
import streamlit as st
import pandas as pd


#pip install streamlit pandas requests
#https://console.groq.com/home Get API Key from Here


# -------------------------
# CONFIG
# -------------------------
GROQ_API_KEY = 'API_KEY_HERE'
SAVE_DIR = r"/Users/serge/OneDrive/Desktop/AIML Specialization/Mandatory Courses/PC AIML - Generative AI Literacy/Projects"
MODEL = "llama-3.1-8b-instant"

os.makedirs(SAVE_DIR, exist_ok=True)

# -------------------------
# STATE INIT
# -------------------------
if "story_intro" not in st.session_state:
    st.session_state.story_intro = ""
if "story_next" not in st.session_state:
    st.session_state.story_next = ""
if "full_story" not in st.session_state:
    st.session_state.full_story = ""
if "selected_choice" not in st.session_state:
    st.session_state.selected_choice = None

# -------------------------
# DATA
# -------------------------
customer_df = pd.read_csv("Customer.csv")
customer_df.head()

# -------------------------
# LLM CALL
# -------------------------
def groq_chat(prompt: str) -> str:
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {'API_KEY_HERE'}",
        "Content-Type": 'application/json',
    }
    payload = {
        "model": MODEL,
        "messages": [
            {"role": "system", "content": "You are a creative storyteller."},
            {"role": "user", "content": prompt},
        ],
        "temperature": 0.7,
    }
    resp = requests.post(url, headers=headers, json=payload)
    if resp.status_code == 200:
        return resp.json()["choices"][0]["message"]["content"]
    return f"❌ Error {resp.status_code}: {resp.text}"

def build_intro_prompt(name, age, gender, profession, theme):
    return f"""
Create a short {theme} story.
The main character is {name}, a {age}-year-old {gender.lower()} who works as a {profession}.
They encounter a mysterious object glowing with unknown energy. Build suspense and lead into a decision point.
"""

# -------------------------
# UI
# -------------------------
st.set_page_config(page_title="Groq Unleashed: Next-Gen Storytelling", layout="centered")
st.title("🧠 Smart Storytelling, Powered by Groq")

# Select customer
customer_ids = customer_df['CustomerID'].tolist()
selected_id = st.selectbox("Select Customer ID:", customer_ids)

row = customer_df[customer_df['CustomerID'] == selected_id].iloc[0]
name = f"Customer{row['CustomerID']}"
age = row['Age']
gender = row['Gender']
profession = row['Profession']

themes = ["Mystery", "Adventure", "Sci-Fi", "Fantasy", "Romance"]
selected_theme = st.radio("Choose a story theme:", ["Mystery", "Adventure", "Sci-Fi", "Fantasy", "Romance"], horizontal=True)

#Start Story button
if st.button("✨ Start Story"):
    prompt = build_intro_prompt(name, age, gender, profession, selected_theme)
    st.session_state.story_intro = groq_chat(prompt)
    st.session_state.story_next = ""
    st.session_state.full_story = st.session_state.story_intro

# Show intro if present
if st.session_state.story_intro:
    st.markdown("### 🌟 The Story Begins")
    st.write(st.session_state.story_intro)

    # Choice radio needs a key to persist between reruns
    options = [
        "1. Touch the glowing object.",
        "2. Observe from a distance.",
        "3. Leave immediately."
    ]
    st.session_state.selected_choice = st.radio(
        "What should the character do next?",
        options,
        index=0 if st.session_state.selected_choice is None else options.index(st.session_state.selected_choice),
        key="choice_radio"
    )

    # Save intro immediately (optional)
    colA, colB, colC = st.columns([1,1,2])
    with colA:
        if st.button("💾 Save Intro Only (.txt)"):
            intro_path = os.path.join(SAVE_DIR, f"{name}_story_intro.txt")
            with open(intro_path, "w", encoding="utf-8") as f:
                f.write(st.session_state.story_intro)
            st.success(f"Saved: {intro_path}")
    with colB:
        st.download_button(
            "⬇️ Download Intro",
            data=st.session_state.story_intro.encode("utf-8"),
            file_name=f"{name}_story_intro.txt",
            mime="text/plain"
        )

    # --- Continue story ---
    if st.button("➡️ Continue Story"):
        next_prompt = (
            f"{st.session_state.story_intro}\n\n"
            f"The character chooses to: {st.session_state.selected_choice}\n\n"
            f"What happens next? Continue the narrative."
        )
        st.session_state.story_next = groq_chat(next_prompt)
        st.session_state.full_story = f"{st.session_state.story_intro}\n\n{st.session_state.story_next}"

# Show continuation + save options
if st.session_state.story_next:
    st.markdown("### 🧩 The Story Continues...")
    st.write(st.session_state.story_next)

    # Auto-save full story locally
    save_path = os.path.join(SAVE_DIR, f"{name}_story.txt")
    try:
        with open(save_path, "w", encoding="utf-8") as f:
            f.write(st.session_state.full_story)
        st.success(f"Full story saved to: {save_path}")
    except Exception as e:
        st.error(f"Could not save file: {e}")

    # Browser download
    st.download_button(
        label="⬇️ Download Full Story",
        data=st.session_state.full_story.encode("utf-8"),
        file_name=f"{name}_story.txt",
        mime="text/plain"
    )